In [1]:
import pandas as pd

from src.pipelines import pipeline_data_collection, pipeline_data_processing

pd.set_option('display.max_columns', 50)

# Data collection

In [2]:
update_tables = dict(despesas=True, 
                     deputados=False)
id_deputados_to_fetch = [220526, 178873, 220527, 220530, 160517, 
                         220528, 204356, 204355, 178871, 220529]
start_date = '2024-05-01'
end_date = '2024-05-01'

pipeline_data_collection(update_tables, id_deputados_to_fetch, start_date, end_date)

ID:220526, 2024-05: 30 registros
ID:178873, 2024-05: 4 registros
ID:220527, 2024-05: 3 registros
ID:220530, 2024-05: 19 registros
ID:160517, 2024-05: 4 registros
ID:220528, 2024-05: 10 registros
ID:204356, 2024-05: 2 registros
ID:204355, 2024-05: 18 registros
ID:178871, 2024-05: 6 registros
ID:220529, 2024-05: 12 registros


# Data preprocessing

In [3]:
update_tables = dict(despesas=True, 
                     deputados=False)

pipeline_data_processing(update_tables)

Table shape: (4827, 28)


# tests

In [4]:
df = pd.read_parquet('data/gold/master_table.parquet')

In [5]:
df.head(1)

,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlosa,numRessarcimento,codLote,parcela,id_deputado,ano_mes,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email,Valor
0,2023,2,COMBUSTÍVEIS E LUBRIFICANTES.,7494371,Nota Fiscal Eletrônica,4,2023-02-01,317169,200.0,http://www.camara.leg.br/cota-parlamentar/nota...,AUTO POSTO GOLDEN GAS 202 LTDA.,30880112000145,200.0,0.0,,1906755,0,220526,2023_02,https://dadosabertos.camara.leg.br/api/v2/depu...,Gilvan da Federal,PL,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.gilvandafederal@camara.leg.br,200.0


In [13]:
def process_gold_monthly_data():
    df = pd.read_parquet('data/gold/master_table.parquet')
    
    group_mes = df.groupby(['ano_mes', 'id_deputado'])
    df_mes = group_mes[['Valor']].sum()
    df_mes['Deputado'] = group_mes['nome'].max()
    df_mes.reset_index(inplace=True)
    
    print(f"Table shape: {df.shape}")
    df_mes.to_parquet('data/gold/monthly_data.parquet') #TODO path in config

process_gold_monthly_data()

Table shape: (4827, 28)


In [14]:
df_mes = pd.read_parquet('data/gold/monthly_data.parquet')

## gasto medio por deputado

In [56]:
periodos = ['2024_05']

df_filtrado = df.loc[df['ano_mes'].isin(periodos)].copy()
mensal_filtrado = df_mes.loc[df_mes['ano_mes'].isin(periodos)].copy()

In [57]:
def calc_avg_spend_from_period(df):
    spend = df.groupby('ano_mes')['Valor'].mean().mean()
    return f"R$ {round(spend, 2)}"

calc_avg_spend_from_period(mensal_filtrado)

'R$ 10548.09'

In [21]:
import plotly.graph_objects as go
import plotly.express as px

## grafico por deputado e tipo de despesa

In [22]:
px.bar(df.loc[df['ano_mes']=='2024_04'], x='nome', y='valorDocumento', color='tipoDespesa')

## deputado que mais gastou por tipo de despesa

'Helder Salomão'

In [63]:
mais_gastou_por_tipo = dict()
tmp = df_filtrado.groupby(['nome', 'tipoDespesa'])['Valor'].sum().reset_index().copy()
for tipo, tmpdf in tmp.groupby('tipoDespesa'):
    result_tipo = tmpdf.loc[tmpdf['Valor']==tmpdf['Valor'].max(), :].copy()
    mais_gastou_por_tipo[tipo] = dict(Deputado=result_tipo['nome'].values[0], 
                                        Valor = result_tipo['Valor'].values[0])
    # tmp[tmp==tmp.max()]:

mais_gastou_por_tipo = pd.DataFrame(mais_gastou_por_tipo).T.sort_values('Valor', ascending=False)
mais_gastou_por_tipo

,Deputado,Valor
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,Amaro Neto,20000.0
PASSAGEM AÉREA - SIGEPA,Da Vitoria,11821.71
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,Messias Donato,10100.0
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,Da Vitoria,7837.5
COMBUSTÍVEIS E LUBRIFICANTES.,Gilvan da Federal,2854.21
"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",Gilvan da Federal,2794.05
TELEFONIA,Paulo Folletto,119.89


In [58]:
mais_gastou_por_tipo = dict()
tmp = df_filtrado.groupby(['nome', 'tipoDespesa'])['Valor'].sum().reset_index().copy()
for tipo_despesa in tmp['tipoDespesa'].unique():
    result = tmp.loc[tmp['tipoDespesa']==tipo_despesa].copy()
    result = result.loc[result['Valor']==result['Valor'].max(), :].copy()
    mais_gastou_por_tipo[tipo_despesa] = dict(Deputado=result['nome'].values[0], 
                                        Valor = result['Valor'].values[0])
    # tmp[tmp==tmp.max()]:


In [59]:
pd.DataFrame(mais_gastou_por_tipo).T

,Deputado,Valor
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,Amaro Neto,20000.0
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,Da Vitoria,7837.5
COMBUSTÍVEIS E LUBRIFICANTES.,Gilvan da Federal,2854.21
PASSAGEM AÉREA - SIGEPA,Da Vitoria,11821.71
"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",Gilvan da Federal,2794.05
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,Messias Donato,10100.0
TELEFONIA,Paulo Folletto,119.89


In [26]:
for deputado, tmpdf in df_filtrado.groupby('deputado'):
    tmpdf.loc[tmpdf['Valor']]


,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlosa,numRessarcimento,codLote,parcela,id_deputado,ano_mes,uri,nome,siglaPartido,uriPartido,siglaUf,idLegislatura,urlFoto,email,Valor
4403,2024,4,COMBUSTÍVEIS E LUBRIFICANTES.,7722512,Nota Fiscal Eletrônica,4,2024-03-22,1256995,243.18,http://www.camara.leg.br/cota-parlamentar/nota...,POSTO BELO,42424137000113,243.18,0.00,,2032251,0,220526,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Gilvan da Federal,PL,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.gilvandafederal@camara.leg.br,243.18
4434,2024,4,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,7731683,Recibos/Outros,1,2024-03-25,30738,2748.66,https://www.camara.leg.br/cota-parlamentar/doc...,REDE BRASILEIRA DE AUTOMOTORES LTDA,39316666000865,2713.00,35.66,,2037050,0,178871,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Evair Vieira de Melo,PP,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.evairvieirademelo@camara.leg.br,2713.00
4459,2024,4,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7723338,Recibos/Outros,1,2024-03-28,4476,2453.54,https://www.camara.leg.br/cota-parlamentar/doc...,Apê Vila Velha Imobiliária e Construções,28926781000169,2450.54,3.00,,2032659,0,204356,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Amaro Neto,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.amaroneto@camara.leg.br,2450.54
4471,2024,4,COMBUSTÍVEIS E LUBRIFICANTES.,7716488,Nota Fiscal Eletrônica,4,2024-03-31,67362,105.15,http://www.camara.leg.br/cota-parlamentar/nota...,REDE QUALITY COMERCIO DE COMBUSTIVEIS LTDA,36900391000571,105.15,0.00,,2030128,0,160517,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Paulo Folletto,PSB,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.paulofolletto@camara.leg.br,105.15
4477,2024,4,COMBUSTÍVEIS E LUBRIFICANTES.,7711604,Nota Fiscal Eletrônica,4,2024-04-01,169524,100.00,http://www.camara.leg.br/cota-parlamentar/nota...,AUTO VOLUME COMERCIO DE DERIVADOS DE PETROLEO ...,25015036000170,100.00,0.00,,2026387,0,220526,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Gilvan da Federal,PL,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.gilvandafederal@camara.leg.br,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4734,2024,4,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7730864,Recibos/Outros,1,2024-05-06,344,680.00,https://www.camara.leg.br/cota-parlamentar/doc...,VITÓRIA TECNOLOGIA DA INFORMAÇÃO LTDA,06069667000127,680.00,0.00,,2036961,0,160517,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Paulo Folletto,PSB,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.paulofolletto@camara.leg.br,680.00
4738,2024,4,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,7730777,Recibos/Outros,1,2024-05-07,06549,12500.00,https://www.camara.leg.br/cota-parlamentar/doc...,ARAUJO RENTACAR EIRELI EPP,07134140000100,12500.00,0.00,,2036567,0,220528,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Dr. Victor Linhalis,PODE,https://dadosabertos.camara.leg.br/api/v2/part...,ES,57,https://www.camara.leg.br/internet/deputado/ba...,dep.dr.victorlinhalis@camara.leg.br,12500.00
4748,2024,4,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,7733119,Nota Fiscal,0,2024-05-08,45,10000.00,https://www.camara.leg.br/cota-parlamentar/doc...,KELLY BARCELOS - MEGA PRODUÇÕES,24469773000180,10000.00,0.00,,2038297,0,220530,2024_04,https://dadosabertos.camara.leg.br/api/v2/depu...,Messias Donato,REPUBLICANOS,https://dadosabertos.camara.leg

In [ ]:
df.groupby('nomeFornecedor')['valorDocumento'].sum().sort_values(ascending=False).head(10)

In [ ]:
df.loc[(df['nome'].str.contains('Linhalis')) & (df['ano_mes']=='2023_12') & (df['valorDocumento'].between(10000, 13000)), ['urlDocumento']].values

In [ ]:
px.line(tmp, x='ano_mes', y='valorDocumento', color='nome')

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
tmp = df.groupby(['id_deputado', 'ano_mes'])[['valorDocumento']].sum()

fig = go.Figure()
for k, tmpdf in df.groupby(['id_deputado']):
    px.line(tmp.loc[dep], y='valorDocumento')
    # fig.add_trace(px.line(tmp.loc[dep], y='valorDocumento'))
fig.show()

# Rascunho

In [ ]:
def pipeline_data_collection():
    info_datasets = {} # config
    reprocessing_status = {}
    for dataset in info_datasets.keys():
        df = collect_data(dataset)
        reprocessing_status[dataset] = get_reprocessing_status()
        if reprocessing_status[dataset]:
            save_data(df)

    save_data(reprocessing_status)

In [ ]:
def pipeline_data_processing():
    
    process_deputados()
    process_despesas()

In [ ]:
pipeline_data_collection()

pipeline_data_processing()

update_website()